In [1]:
!pip install -q pandas
!pip install dask
!pip install pyarrow
!pip install fastparquet phonetics fuzzywuzzy jellyfish putils -q
!pip install seaborn -q 
!pip install matplotlib -q 

import pandas as pd
!pip install recordlinkage
import recordlinkage as rl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd
import psutil
import psutil
import recordlinkage as rl
from recordlinkage.index import SortedNeighbourhood
import pickle

def get_matches(df, cols):
  df = df.reset_index()
  df = merge_tid_lmk(df)
  for col in cols:
    df = df[df[col] != 0]
  return df

# Function to report RAM usage
def report_ram_usage():
    process = psutil.Process()
    ram_usage = process.memory_info().rss / (1024 ** 2)  # in MB
    print(f"Current RAM usage: {ram_usage:.2f} MB")

# Report RAM before starting
report_ram_usage()

# get cpu number
import multiprocessing
multiprocessing.cpu_count()

# get RAM
!cat /proc/meminfo

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Current RAM usage: 302.41 MB
MemTotal:       258379312 kB
MemFree:        255015456 kB
MemAvailable:   255303108 kB
Buffers:            2704 kB
Cached:          2152408 kB
SwapCached:            0 kB
Active:           501176 kB
Inactive:        2418840 kB
Active(anon):        608 kB
Inactive(anon):   765188 kB
Active(file):     500568 kB
Inactive(file):  1653652 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                16 kB
Writeback:             0 kB
AnonPages:        765084 kB
Mapped:           299792 kB
Shmem:               892 kB
KReclaimable:     113528 kB
Slab:             227592 kB
SReclaimable:     113528 kB
SUnreclaim:       114064 kB
KernelStack:       11456 kB
PageTa

In [2]:
multiprocessing.cpu_count()

32

### For loop to get all

In [3]:
names = ['11-01', '01-91', '91-81', '81-61', '61-51']
years = ['1851', '1861', '1881', '1891', '1901', '1911']
toys_with_features = {}
candidates_all_year = {}
for name in names:
    # reading candidates
    filename = f'../../output/task1/{name}_candidates.sav'
    candidates_all_year[name] = pickle.load(open(filename, 'rb'))
    
# reading toy datasets with features
# years = ['1851', '1861', '1881', '1891', '1901', '1911']
for year in years:
    df = pd.read_parquet(f'../../data_toy/task1/{year}_anynomised.parquet').rename(columns = {'ConParID_18511911':'parID', 'CONPARID_birth':'birth_parID', 'sex':'sex'}).set_index('recid')
    toys_with_features[year] = df
# test consistency
assert toys_with_features['1911'].index.intersection(candidates_all_year['11-01'].get_level_values(0)).shape[0] == toys_with_features['1911'].shape[0]
assert toys_with_features['1901'].index.intersection(candidates_all_year['11-01'].get_level_values(1)).shape[0] == toys_with_features['1901'].shape[0]
assert toys_with_features['1851'].index.intersection(candidates_all_year['61-51'].get_level_values(1)).shape[0] == toys_with_features['1851'].shape[0]

In [4]:
toys_with_features['1851'].columns

Index(['pname', 'oname', 'sname', 'pname_soundex', 'sname_soundex',
       'pname_metaphone', 'sname_metaphone', 'address', 'sname_pop_metaphone',
       'dateofbirth', 'sex', 'birth_parID', 'parID', 'pname_mom_soundex',
       'sname_mom_soundex', 'pname_pop_soundex', 'sname_pop_soundex',
       'pname_sp_soundex', 'sname_sp_soundex', 'pname_mom_metaphone',
       'sname_mom_metaphone', 'pname_pop_metaphone', 'pname_sp_metaphone',
       'sname_sp_metaphone', '_51'],
      dtype='object')

In [5]:
%%time
import psutil
from Utils.sex_1_2_9 import sex_1_2_9
from Utils.geo import GeoTable
# Initialize the comparer directly in the script
print("Initializing comparer...")
# comparer = rl.Compare(n_jobs = -1)  # Utilizing all available CPUs
comparer = rl.Compare()

# cluster_features = ['pname', 'oname', 'sname', 'pname_soundex', 'sname_soundex', 'pname_metaphone', 'sname_metaphone', 'address', 'dateofbirth', 'par_witin_10km', 'birth_par_witin_10km', 'sname_pop_metaphone', 'sex']

# Define similarity methods for each attribute pair
str_methods = {
    'pname': 'jarowinkler',
    'oname': 'jarowinkler',
    'sname': 'jarowinkler',
    'pname_soundex': 'jarowinkler',
    'sname_soundex': 'jarowinkler',
    'pname_metaphone': 'jarowinkler',
    'sname_metaphone': 'jarowinkler',
    'address': 'levenshtein',
    # new ones in the 1st iteration
    'sname_pop_metaphone': 'jarowinkler'
}

# Add string comparison methods to comparer
for attr, method in str_methods.items():
    comparer.string(attr, attr, method=method)

# Add numeric comparison for the 'dateofbirth' column
comparer.numeric('dateofbirth', 'dateofbirth', method = 'gauss', offset = 0, scale = 5) 
# sex
comparer.add(sex_1_2_9('sex', 'sex', 'sex'))

# geo
buffer_10k = pd.read_csv('../../data_toy/task1/Buffer_10k.csv')
pairs_within_10km = {pair['conparid_1']:pair['conparid_2'] for pair in buffer_10k.to_dict(orient = 'records')}

comparer.add(GeoTable('parID', 'parID',  pairs_within_10km, label = 'par_witin_10km'))
comparer.add(GeoTable('birth_parID', 'birth_parID', pairs_within_10km, label = 'birth_par_witin_10km'))

# Report RAM after adding methods
report_ram_usage()

Initializing comparer...
Current RAM usage: 4667.95 MB
CPU times: user 330 ms, sys: 84.4 ms, total: 415 ms
Wall time: 538 ms


In [9]:
# right = ['1851', '1861', '1881', '1891', '1901']
# left = ['1861', '1881', '1891', '1901', '1911']

right = ['1861', '1881', '1891', '1901']
left = ['1881', '1891', '1901', '1911']

cols_to_compare = list(str_methods.keys())
# add dateofbirth sex birth_parID parID to cols_to_compare
to_add = ['dateofbirth', 'sex', 'birth_parID', 'parID']
cols_to_compare.extend(to_add)

for l_y, r_y in zip(left, right):
    compare_name = l_y[-2:] + '-' + r_y[-2:]
    result = comparer.compute(candidates_all_year[compare_name], toys_with_features[l_y], toys_with_features[r_y])
    result.to_parquet(f'../../output/task1/{compare_name}_compared.parquet')

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


/opt/apps/hand/miniconda/23.5.2/lib/python3.11/site-packages/pandas/io/parquet.py:189: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


/opt/apps/hand/miniconda/23.5.2/lib/python3.11/site-packages/pandas/io/parquet.py:189: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


/opt/apps/hand/miniconda/23.5.2/lib/python3.11/site-packages/pandas/io/parquet.py:189: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


/opt/apps/hand/miniconda/23.5.2/lib/python3.11/site-packages/pandas/io/parquet.py:189: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


In [11]:
!ls -lah ../../output/task1/**parquet

-rw-r--r-- 1 xiet13 cluster-users 181M Mar  5 12:32 ../../output/task1/01-91_compared.parquet
-rw-r--r-- 1 xiet13 cluster-users 181M Mar  5 12:41 ../../output/task1/11-01_compared.parquet
-rw-r--r-- 1 xiet13 cluster-users 177M Mar  4 16:10 ../../output/task1/61-51_compared.parquet
-rw-r--r-- 1 xiet13 cluster-users 177M Mar  5 12:14 ../../output/task1/81-61_compared.parquet
-rw-r--r-- 1 xiet13 cluster-users 181M Mar  5 12:23 ../../output/task1/91-81_compared.parquet


In [ ]:
# result.birth_par_witin_10km.value_counts(), result.par_witin_10km.value_counts()

In [ ]:
stopper

## Record Linkage

In [ ]:
# from dask.distributed import LocalCluster 
# import dask.dataframe as dd

# # client = LocalCluster(n_workers = 32)
# client = LocalCluster(n_workers = 8, threads_per_worker=4)

In [ ]:
# import psutil
# import recordlinkage as rl
# from recordlinkage.index import SortedNeighbourhood

### Initialize the indexer

In [ ]:
# !ls -lha ../Output/temp/{name}/name_soundex_and_district_birth_ID

### Read indices 

In [ ]:
# import pickle
# # name = ""
# name = "91-81"
# read_from_multiple = True

# if read_from_multiple:
#     # initiate an empty MultiIndex with recid_1 and recid_2 as index
#     # candidates = pd.MultiIndex(levels=[[], []], codes=[[], []], names=['recid_1', 'recid_2'])
#     candidates = []
#     # all files in directory
#     import os
#     for file in os.listdir(f"../Output/temp/{name}/name_soundex_and_district_birth_ID"):
#         if file.endswith(".sav"):
#             candidates.append(pickle.load(open(os.path.join(f"../Output/temp/{name}/name_soundex_and_district_birth_ID", file), 'rb')))
#             print(f"loaded {file}")
#     # union is too slow
#     # get them rogether
#     candidates = pd.concat([c.to_frame().reset_index(drop = True).drop_duplicates(subset = ['recid_1', 'recid_2']) for c in candidates], ignore_index=True)
#     candidates = candidates.set_index(['recid_1', 'recid_2']).index
# else:
#     filename = f'../Output/{name}_pool_idx.sav'
#     # pickle.dump(candidates, open(filename, 'wb'))
#     candidates = pickle.load(open(filename, 'rb'))

# # check if candidates is unique
# candidates.is_unique # 1m 7.3s


### Reread with comparing columns

In [ ]:
# to_use = ['recid','pname',
#  'oname',
#  'sname',
#  'pname_soundex',
#  'sname_soundex',
#  'pname_metaphone',
#  'sname_metaphone',
#  'address',
#  'sname_pop_metaphone',
#  'dateofbirth',
#  'sex',
#  'CONPARID_birth',
#  'ConParID_18511911',
#  ]

# _1881 = dd.read_parquet('../data_toy/task1/1881_anynomised.parquet', columns = to_use).compute(scheduler = 'processes') # unique
# _1881 = _1881.drop_duplicates(subset = ['recid'])
# _1891 = dd.read_parquet('../data_toy/task1/1881_anynomised.parquet', columns = to_use).compute(scheduler = 'processes') # unique
# _1891 = _1891.drop_duplicates(subset = ['recid'])

# # renaming
# _1881 = _1881.rename(columns = {'ConParID_18511911':'parID', 'CONPARID_birth':'birth_parID', 'sex':'sex'}).set_index('recid')
# _1891 = _1891.rename(columns = {'ConParID_18511911':'parID', 'CONPARID_birth':'birth_parID', 'sex':'sex'}).set_index('recid')

In [ ]:
# metrics for pool quality
# candidates.get_level_values(0).unique() * candidates.get_level_values(1).unique() / candidates.shape[0] # 242155
# 11300570 * 11068409 / 516524465 # 242155

# 516524465 / (19828561 * 17711037) # 1.4708070690279348e-06: proportion to full indexing

In [ ]:
# _1891.index.unique() # 19828561
# _1851.index.unique() # 17711037
# candidates.get_level_values(0).unique() # 11300570 (57.00% of 1861)
# candidates.get_level_values(1).unique() # 11068409 (62.49% of 1851)

# # subset candidates with get_level_values(0) in 1861.index and get_level_values(1) in 1851.index
# candidates_sub = candidates[candidates.get_level_values(0).isin(_1891.index) & candidates.get_level_values(1).isin(_1851.index)]

In [ ]:
# check if candidates is unique
# candidates.is_unique
# check if all candidates is within 1851 and 1861
assert all(candidates.get_level_values(0).isin(_1891.index) & candidates.get_level_values(1).isin(_1891.index))

ddf__1881 = _1881
ddf__1891 = _1891

### Comparer Design

In [ ]:
# %%time
# import psutil
# from Utils.sex_1_2_9 import sex_1_2_9
# from Utils.geo import GeoTable
# # Initialize the comparer directly in the script
# print("Initializing comparer...")
# # comparer = rl.Compare(n_jobs = -1)  # Utilizing all available CPUs
# comparer = rl.Compare()

# # cluster_features = ['pname', 'oname', 'sname', 'pname_soundex', 'sname_soundex', 'pname_metaphone', 'sname_metaphone', 'address', 'dateofbirth', 'par_witin_10km', 'birth_par_witin_10km', 'sname_pop_metaphone', 'sex']

# # Define similarity methods for each attribute pair
# str_methods = {
#     'pname': 'jarowinkler',
#     'oname': 'jarowinkler',
#     'sname': 'jarowinkler',
#     'pname_soundex': 'jarowinkler',
#     'sname_soundex': 'jarowinkler',
#     'pname_metaphone': 'jarowinkler',
#     'sname_metaphone': 'jarowinkler',
#     'address': 'levenshtein',
#     # new ones in the 1st iteration
#     'sname_pop_metaphone': 'jarowinkler'
# }

# # Add string comparison methods to comparer
# for attr, method in str_methods.items():
#     comparer.string(attr, attr, method=method)


# # Define ID columns for exact comparison
# # ID_cols = [
# #     'subdist_ID', 'district_ID', 'county_ID', 'CONPARID_birth',
# #     'subdist_birth_ID', 'district_birth_ID', 'county_birth_ID',
# #     'recid_mom', 'recid_pop', 'recid_sp'
# # ]
# # ID_cols = [
# #     'recid_mom', 'recid_pop', 'recid_sp' # make features for families, or use later 
# # ]
# # # Add exact comparison methods to comparer for ID columns
# # for attr in ID_cols:
# #     comparer.exact(attr, attr)


# # Add numeric comparison for the 'dateofbirth' column
# comparer.numeric('dateofbirth', 'dateofbirth', method = 'gauss', offset = 0, scale = 5) 
# # sex
# comparer.add(sex_1_2_9('sex', 'sex', 'sex'))

# # geo
# buffer_10k = pd.read_csv('../Census_samples/Buffer_10k.csv')
# pairs_within_10km = {pair['conparid_1']:pair['conparid_2'] for pair in buffer_10k.to_dict(orient = 'records')}

# comparer.add(GeoTable('parID', 'parID',  pairs_within_10km, label = 'par_witin_10km'))
# comparer.add(GeoTable('birth_parID', 'birth_parID', pairs_within_10km, label = 'birth_par_witin_10km'))
# # comparer.add(GeoTable('parID', 'parID', pairs_within_10km,  'par_witin_10km'))
# # comparer.add(GeoTable('birth_parID', 'birth_parID', pairs_within_10km, 'birth_par_witin_10km'))

# # comparer.numeric('sex', 'sex', method = 'lin', offset = 0, scale = 5)  
# # # for {0:m, 5:unknown, 10:female} sex value design: (0,0) get 1, (0,5) get 0.5, (0,10) get 0. 
# # # Imprecise account for (5,5) case, where should be 0.5, but get 1.

# # https://www.elastic.co/guide/en/elasticsearch/guide/current/decay-functions.html

# # Custom comparer: Add checking if CONPARIDs are within 10 km
# # comparer.add(WithinDist('ConParID_18511911', 'ConParID_18511911', pairs_within_10km))
# # comparer.add(WithinDist('CONPARID_birth', 'CONPARID_birth', pairs_within_10km))
    

# # Report RAM after adding methods
# report_ram_usage()
# _1881

### Perform Comparing

In [ ]:
# cols_to_compare = list(str_methods.keys())
# # add dateofbirth sex birth_parID parID to cols_to_compare
# to_add = ['dateofbirth', 'sex', 'birth_parID', 'parID']
# cols_to_compare.extend(to_add)
# result = comparer.compute(candidates, df1, df2, n_jobs=-1)

#### Cluster for comparing

In [ ]:
# from dask.distributed import LocalCluster, Client
# import dask.dataframe as dd

# # client = LocalCluster(n_workers = 32)
# # client = LocalCluster(n_workers = 4, threads_per_worker = 8,  interface='lo')
# clinet = Client(n_workers = 4, threads_per_worker = 8)

In [ ]:
# %%time
# import os
# import pandas as pd
# import dask
# from dask.diagnostics import ProgressBar

# # Perform the actual comparisons and store the result
# print("Performing comparisons...")

# def compute_and_save_partition(comparer, candidates, df1, df2, partition_path):
#     if os.path.exists(partition_path):
#         print(f"Skipping already processed partition: {partition_path}")
#         return
#     # Perform the comparison
#     print(f"Computing partition: {partition_path}")
#     result = comparer.compute(candidates, df1, df2, n_jobs=-1)
#     # Save the result
#     result.to_parquet(partition_path)
#     print(f"Saved {partition_path}")
#     # Report RAM usage
#     report_ram_usage()
#     # Return the result
#     return result


# # def parallel_compute_and_save(comparer, candidates, df1, df2, output_dir, partition=1000000):
# #     tasks = []
# #     total_candidates = candidates.shape[0]
# #     for i in range(0, total_candidates, partition):
# #         end = min(i + partition, total_candidates)
# #         partition_path = 
# #         print(f"Sliced {end} - {i}\n")
# #         tasks.append(compute_and_save_partition(comparer, candidates[start:end].drop_duplicates(), df1, df2, partition_path))
# #         print(f"Finished {i} - {end}\n")
# #     return tasks

# # Ensure the output directory exists
# output_dir = f"../Output/temp/{name}_compare_corrected_13/"
# os.makedirs(output_dir, exist_ok=True)

# partition=1000000

# # Applying the delayed function with saving and loading
# tasks = [dask.delayed(compute_and_save_partition)(comparer, candidates[i:i+partition], ddf__1891[cols_to_compare], ddf__1881[cols_to_compare], 
#             os.path.join(output_dir, f'partition_{i}_{i+partition}.parquet'))
#             for i in range(0, candidates.shape[0], partition)]

# # Compute the result
# with dask.diagnostics.ProgressBar():
#     dask.compute(*tasks)

# # Compute all tasks in parallel
# # dask.compute(*tasks)

In [ ]:
# # aa
# import os
# import pandas as pd
# import dask
# from dask.diagnostics import ProgressBar

# # Perform the actual comparisons and store the result
# print("Performing comparisons...")

# # @dask.delayed
# def compute_and_save_partition(comparer, candidates, df1, df2, start, end, partition_path):
#     # Check if the partition is already computed and saved
#     try:
#         # Check if the partition is already computed and saved
#         if os.path.exists(partition_path):
#             # Load the partition from the Parquet file
#             return pd.read_parquet(partition_path)
#         else:
#             # Compute the partition
#             candidate_slice = candidates[start:end].drop_duplicates() # len(candidates) == 1000,000
#             # if len(candidate_slice) != 1000000:
#             #     raise ValueError(f"candidate_slice has incorrect number of rows: {len(candidate_slice)}")
#             # if len(df1) != 1000000 or len(df2) != 1000000:
#             #     raise ValueError(f"Mismatch in number of rows. df1: {len(df1)}, df2: {len(df2)}")

#             # partition_result = comparer.compute(candidate_slice, df1, df2, n_jobs=-1)
#             partition_result = comparer.compute(candidate_slice, df1, df2)
#             # Error details: Length mismatch: Expected axis has 1000012 elements, new values have 1000000 elements

#             # Save the computed partition to a Parquet file
#             partition_result.to_parquet(partition_path)
#             return partition_result
#     except ValueError as e:
#         # Log detailed error information
#         print(f"Error processing partition {partition_path.split('_')[1:-1]}: start={start}, end={end}, candidate_slice: {len(candidate_slice)}")
#         print(f"Error details: {e}")
#         raise

# def parallel_compute_and_save(comparer, candidates, df1, df2, output_dir, partition=1000000):
#     result = []
#     total_candidates = candidates.shape[0]
#     for i in range(0, total_candidates, partition):
#         end = min(i + partition, total_candidates)
#         partition_path = os.path.join(output_dir, f'partition_{i}_{end}.parquet')
#         print(f"Sliced {end} - {i}\n")
#         result.append(compute_and_save_partition(comparer, candidates, df1, df2, i, end, partition_path))
#     return result

# # Ensure the output directory exists
# output_dir = f"../Output/temp/{name}_compare_corrected_13/"
# os.makedirs(output_dir, exist_ok=True)

# # # Applying the delayed function with saving and loading

# # # Compute the result
# # with dask.diagnostics.ProgressBar():
# #     result = delayed_result.compute()

# with dask.diagnostics.ProgressBar():
#     result = parallel_compute_and_save(comparer, candidates, _1891[cols_to_compare], _1881[cols_to_compare], output_dir)

# final_result = pd.concat(result)
# # Update the columns of the result dataframe
# final_result.columns = ['pname', 'oname', 'sname', 'pname_soundex','sname_soundex',
#     'pname_metaphone', 'sname_metaphone', 'address','sname_pop_metaphone', 'dateofbirth', 'sex', 'par_witin_10km', 'birth_par_witin_10km']
# final_result = final_result.reset_index()

In [ ]:
final_result = dd.read_parquet(output_dir)
final_result.columns = ['pname', 'oname', 'sname', 'pname_soundex','sname_soundex',
    'pname_metaphone', 'sname_metaphone', 'address','sname_pop_metaphone', 'dateofbirth', 'sex', 'par_witin_10km', 'birth_par_witin_10km']
final_result = final_result.reset_index()

In [ ]:
# get unique recid_1
len(candidates.get_level_values(0).unique()), len(candidates.get_level_values(1).unique())

In [ ]:
# !ls -lha ../Output/temp/{name}_compare_corrected_13/
# count the number of files in the directory
!ls -l ../Output/temp/{name}_compare_corrected_13/ | wc -l